In [153]:
# Main code written by Morten Hjorth-Jensen
# Modified by Joseph P.Vera and Luis Jimenez

# Common imports
import os

# Where to save the figures and data files
PROJECT_ROOT_DIR = "Results"
FIGURE_ID = "Results/FigureFiles"
DATA_ID = "Results/VMCHarmonic"

if not os.path.exists(PROJECT_ROOT_DIR):
    os.mkdir(PROJECT_ROOT_DIR)

if not os.path.exists(FIGURE_ID):
    os.makedirs(FIGURE_ID)

if not os.path.exists(DATA_ID):
    os.makedirs(DATA_ID)

def image_path(fig_id):
    return os.path.join(FIGURE_ID, fig_id)

def data_path(dat_id):
    return os.path.join(DATA_ID, dat_id)

def save_fig(fig_id):
    plt.savefig(image_path(fig_id) + ".png", format='png')

outfile = open(data_path("VMCHarmonic.dat"),'w')

In [157]:
%matplotlib inline
from math import exp, sqrt
from random import random, seed, normalvariate
import numpy as np
import matplotlib.pyplot as plt
from decimal import *

# Trial wave function for the Harmonic oscillator in generalized form (Generalized equation)
#betha = 1
def WaveFunction(r,alpha):
    sum = 0
    for i in range(N):
        for j in range(D):
            sum += r[i,j]*r[i,j]
    return exp(-alpha*sum)

# Local energy  for the Harmonic oscillator in generalized form (Generalized equation)
def LocalEnergy(r,alpha):
    sum1 = 0
    for i in range(N):
        for j in range(D):
            sum1 += r[i,j]*r[i,j] 
    return (0.5 - 2*alpha*alpha)*sum1 + N*D*alpha

# Derivate of wave function ansatz as function of variational parameters
def DerivativeWFansatz(r,alpha):
    sum = 0
    for i in range(N):
        for j in range(D):
            sum += r[i,j]*r[i,j]
    return -1*sum

# Drift force (Generalized equation)
def QuantumForce(r,alpha):
    qforce = np.zeros((N,D), np.double)
    sum2 = 0
    for i in range(N):
        for j in range(D):
            sum2 += r[i,j] 
    qforce[i,:] = -4*alpha*sum2
    return qforce

In [158]:
# The Monte Carlo sampling with the Metropolis algorithm
# The argument types will be inferred by Numba when the function is called.
def EnergyMinimization(alpha):

    NumberMCcycles= 10000  # 7 zeros
    Dif = 0.5
    StepSize = 1.0
    # positions
    PositionOld = np.zeros((N,D), np.double)
    PositionNew = np.zeros((N,D), np.double)
    # Quantum force
    QuantumForceOld = np.zeros((N,D), np.double)
    QuantumForceNew = np.zeros((N,D), np.double)

    # seed for random number generator
    seed()
    energy = 0.0
    DeltaE = 0.0
    EnergyDer = np.zeros((2), np.double)
    DeltaPsi = np.zeros((2), np.double)
    DerivativePsiE = np.zeros((2), np.double)
    #Initial position
    for i in range(N):
        for j in range(D):
            PositionOld[i,j] = normalvariate(0.0,1.0)*sqrt(StepSize)
    wfold = WaveFunction(PositionOld,alpha)
    QuantumForceOld = QuantumForce(PositionOld,alpha)
        
    #Loop over MC MCcycles
    for MCcycle in range(NumberMCcycles):
        #Trial position 
        for i in range(N):
            for j in range(D):
                PositionNew[i,j] = PositionOld[i,j] + normalvariate(0.0,1.0)*sqrt(StepSize) + QuantumForceOld[i,j]*StepSize*Dif
            wfnew = WaveFunction(PositionNew,alpha)
            QuantumForceNew = QuantumForce(PositionNew,alpha)
            GreensFunction = 0.0
            for j in range(D):
                GreensFunction += 0.5*(QuantumForceOld[i,j]+QuantumForceNew[i,j])*(Dif*StepSize*0.5*(QuantumForceOld[i,j]-QuantumForceNew[i,j])- PositionNew[i,j]+PositionOld[i,j])
            GreensFunction = exp(GreensFunction)
            ProbabilityRatio = GreensFunction*wfnew**2/wfold**2
            #Metropolis-Hastings (Morkov chain) test to see whether we accept the move
            if random() <= ProbabilityRatio:
               for j in range(D):
                   PositionOld[i,j] = PositionNew[i,j]
                   QuantumForceOld[i,j] = QuantumForceNew[i,j]
               wfold = wfnew
        DeltaE = LocalEnergy(PositionOld,alpha)
        DerPsi = DerivativeWFansatz(PositionOld,alpha)
        DeltaPsi += DerPsi
        energy += DeltaE
        DerivativePsiE += DerPsi*DeltaE
    #We calculate mean, variance and error
    energy /= NumberMCcycles
    DerivativePsiE /= NumberMCcycles
    DeltaPsi /= NumberMCcycles
    EnergyDer  = 2*(DerivativePsiE-DeltaPsi*energy)
    return energy, EnergyDer

In [159]:
import time
inicio = time.time()
#############################################################################################################################################
# Values for N and D can be changed. N take values {1,2,3,...,500,...} and D is given by {1,2,3}
N=1  #Number of particles
D=3  #Dimensions

alpha = 0.1
# Set up iteration using gradient descent method
Energy = 0
EDerivative = np.zeros((2), np.double)
eta = 0.01
Niterations = 50
# 
for iter in range(Niterations):
    Energy, EDerivative = EnergyMinimization(alpha)
    alphagradient = EDerivative[0]
#    betagradient = EDerivative[1]
    alpha -= eta*alphagradient
#    beta -= eta*betagradient 
print("---------------------------------------------------------------")
print("---------------------------------------------------------------")
print("Alpha value =",alpha)
print("---------------------------------------------------------------")
print("---------------------------------------------------------------")
print("Energy =",Energy)
print("---------------------------------------------------------------")
print("---------------------------------------------------------------")
print("Energy derivate =",EDerivative[0])
print("---------------------------------------------------------------")
print("---------------------------------------------------------------")
#############################################################################################################################################
fin = time.time()
print("---------------------------------------------------------------")
print("---------------------------------------------------------------")
print("CPU time =",fin-inicio, "seconds")
print("CPU time =",(fin-inicio)/60, "minutes")
print("CPU time =",(fin-inicio)/3600, "hours")
print("---------------------------------------------------------------")
print("---------------------------------------------------------------")

---------------------------------------------------------------
---------------------------------------------------------------
Alpha value = 0.499952644441677
---------------------------------------------------------------
---------------------------------------------------------------
Energy = 1.500004460117673
---------------------------------------------------------------
---------------------------------------------------------------
Energy derivate = -0.0002731579515877769
---------------------------------------------------------------
---------------------------------------------------------------
---------------------------------------------------------------
---------------------------------------------------------------
CPU time = 7.431271076202393 seconds
CPU time = 0.12385451793670654 minutes
CPU time = 0.0020642419656117755 hours
---------------------------------------------------------------
---------------------------------------------------------------
